In [1]:
import json
import io
import pandas as pd
from eval.chat_benchmarks.SimpleQA.judge_prompt import JUDGE_PROMPT
from openai import OpenAI
client = OpenAI()

def create_request_data(id, content):
    data_template = {"custom_id": f"request-{id}", 
                    "method": "POST", 
                    "url": "/v1/chat/completions", 
                    "body": {"model": "gpt-4.1-2025-04-14", 
                            "messages": [{"role": "user", "content": content}]}}
    return data_template

def read_data(path):
    data = json.load(open(path))
    # data.keys()
    data_key = list(data['results'].keys())[0]
    data_ = data['results'][data_key]['examples']
    df = pd.DataFrame(data_)
    return df


In [2]:
path = '/home/dongkeun/ckl_r1/evalchemy/logs/final/nonambigqa_val_1k/confidence_no_trigger/deepseek-ai__DeepSeek-R1-Distill-Qwen-32B/results_2025-04-19T15-40-29.387192.json'
df = read_data(path)
df.head(1)

,question,answers,model_think,model_output,model_confidence,grade_letter,correct,not_attempted
0,Who is the author of al capone does my shirts?,['Gennifer Choldenko'],"Okay, so I need to figure out who wrote the bo...",\n\n**Answer**: Gennifer Choldenko\n**Confiden...,Better than even,A,1,0


In [3]:
from prompts import PROMPT_UNCERTAINTY, PROMPT_LINEAR

mode = 'uncertainty'

if mode == 'linear':
    PROMPT = PROMPT_LINEAR
elif mode == 'uncertainty':
    PROMPT = PROMPT_UNCERTAINTY
else:
    raise ValueError("Invalid mode selected.")


def apply_request_data(example):
    prompt = PROMPT.format(question=example['question'], model_think=example['model_think'])
    id = f'{mode}_{example.name}'
    return create_request_data(id, prompt)

request_data = df.apply(apply_request_data, axis=1)
request_data.to_json('request_data/request_data.jsonl', orient='records', lines=True)

In [4]:

# Upload to OpenAI as if it were a file
batch_input_file = client.files.create(
    file=open("request_data/request_data.jsonl", "rb"),
    purpose="batch"
)

batch_input_file_id = batch_input_file.id
batch_info = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": f'{mode}_{path}'
    }
)


In [5]:
print(batch_info.id)

batch_680d60f68b808190afa308d3e7693fdc
